# NYC Real Estate Permits

In [54]:
!pip install sodapy

In [249]:
!pip install sqlalchemy

In [1]:
import mysql.connector 
import config_aws
import requests
import time  # This is just to be able to make multiple calls to the API in spaced out time intervals
import string # For the remove_special_chars function
import pandas as pd
from sodapy import Socrata
import sqlalchemy as db
from statsmodels.stats.multicomp import (pairwise_tukeyhsd,
                                         MultiComparison)
from statsmodels.formula.api import ols
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns


sns.set(style='white',palette='vlag')
%matplotlib inline

In [ ]:
config_aws.api_key
config_aws.app_token
config_aws.username

In [35]:
#instantiate the connection and table

engine = db.create_engine(f'mysql+mysqlconnector://{config_aws.user}:{config_aws.pw}@{config_aws.host}/re_permits')

connection = engine.connect()
metadata=db.MetaData()


# Data Exploration

Data Source: https://data.cityofnewyork.us/Housing-Development/DOB-Permit-Issuance/ipu4-2q9a

The Department of Buildings (DOB) issues permits for construction and demolition activities in the City of New York. The construction industry must submit an application to DOB with details of the construction job they would like to complete. The primary types of application, aka job type, are: New Building, Demolition, and Alterations Type 1, 2, and 3. Each job type can have multiple work types, such as general construction, boiler, elevator, and plumbing. Each work type will receive a separate permit. (See the DOB Job Application Filings dataset for information about each job application.) Each row/record in this dataset represents the life cycle of one permit for one work type. The dataset is updated daily with new records, and each existing record will be updated as the permit application moves through the approval process to reflect the latest status of the application.

In [6]:
# url = 'https://data.cityofnewyork.us/resource/ipu4-2q9a.json'
# response = requests.get(url)
# response

<Response [200]>

In [131]:
# client = Socrata("https://data.cityofnewyork.us/resource/ipu4-2q9a.json", config_aws.app_token)
client = Socrata('data.cityofnewyork.us',
                 config_aws.app_token)


results = client.get("ipu4-2q9a", limit=1000, offset=10000)
df = pd.DataFrame.from_records(results)
    


In [132]:
df.head()

,bin__,bldg_type,block,borough,city,community_board,dobrundate,expiration_date,filing_date,filing_status,...,site_safety_mgr_business_name,site_safety_mgr_s_first_name,site_safety_mgr_s_last_name,special_district_1,state,street_name,superintendent_business_name,superintendent_first___last_name,work_type,zip_code
0,1012151,2,00642,MANHATTAN,ALPHARETTA,102,2019-11-07T00:00:00.000,2020-11-05T00:00:00.000,2019-11-06T00:00:00.000,RENEWAL,...,NaN,NaN,NaN,NaN,GA,JANE STREET,NaN,NaN,EQ,10014
1,3425883,2,01899,BROOKLYN,BROOKLYN,303,2019-11-07T00:00:00.000,2020-11-05T00:00:00.000,2019-11-06T00:00:00.000,RENEWAL,...,NaN,NaN,NaN,NaN,NY,FRANKLIN AVENUE,CARIBEAN PROJECT DESIGNER,NGUYEN CESPEDES,NaN,11205
2,3050541,2,01801,BROOKLYN,ROCKVILLE CENTR,303,2019-11-07T00:00:00.000,2020-03-19T00:00:00.000,2019-09-12T00:00:00.000,INITIAL,...,NaN,NaN,NaN,NaN,NY,LEXINGTON AVE,PHILIP FERSTLER,PHILIP FERSTLER,NaN,11221
3,1077549,2,00367,MANHATTAN,NEW YORK,103,2019-11-07T00:00:00.000,2020-11-05T00:00:00.000,2019-11-06T00:00:00.000,INITIAL,...,NaN,NaN,NaN,NaN,NY,FDR DRIVE,NaN,NaN,PL,10009
4,2113012,2,03272,BRONX,BRONX,206,2019-11-07T00:00:00.000,2020-08-01T00:00:00.000,2019-11-06T00:00:00.000,INITIAL,...,NaN,NaN,NaN,NaN,NY,DR KAZIMIROFF BOULEVARD,NaN,NaN,OT,10458


In [121]:
df.isna().sum()

bin__                                 0
bldg_type                             0
block                                 0
borough                               0
city                                  0
community_board                       1
dobrundate                            0
expiration_date                       0
filing_date                           0
filing_status                         0
gis_census_tract                      5
gis_council_district                  5
gis_latitude                          5
gis_longitude                         5
gis_nta_name                          5
hic_license                         995
house__                               0
issuance_date                         0
job__                                 0
job_doc___                            0
job_start_date                        0
job_type                              0
lot                                   0
non_profit                            3
oil_gas                             999


In [122]:
df.dtypes

bin__                               object
bldg_type                           object
block                               object
borough                             object
city                                object
community_board                     object
dobrundate                          object
expiration_date                     object
filing_date                         object
filing_status                       object
gis_census_tract                    object
gis_council_district                object
gis_latitude                        object
gis_longitude                       object
gis_nta_name                        object
hic_license                         object
house__                             object
issuance_date                       object
job__                               object
job_doc___                          object
job_start_date                      object
job_type                            object
lot                                 object
non_profit 

In [123]:
def clean_data(df1):
    df1.drop(columns=['hic_license','oil_gas', 'permittee_s_other_title', 'site_safety_mgr_business_name',
                       'site_safety_mgr_s_first_name', 'site_safety_mgr_s_last_name', 'special_district_1',
                       'superintendent_business_name','superintendent_first___last_name', 'site_fill', 
                      'permittee_s_business_name','permittee_s_first_name', 'permittee_s_last_name', 'owner_s_first_name',
                      'owner_s_house__', 'owner_s_house_street_name', 'owner_s_last_name', 'owner_s_phone__', 'permit_subtype',
                     'street_name'], 
                     inplace=True)

    df1['residential'].fillna('No', inplace=True)
    
    df1['work_type'].fillna('NA', inplace=True)
    
#     df1['permit_subtype'].fillna('NA', inplace=True)
    
    df1['self_cert'].fillna('NA', inplace=True)
    
    df1.dropna(axis=0, inplace=True)
    
    df = df1.to_dict(orient = 'records')
    
    return df

cleaned_dict = clean_data(df)

In [124]:
# cleaned_dict.shape

In [125]:
# cleaned_dict.isna().sum()

In [126]:
cleaned_dict[0].keys()

dict_keys(['bin__', 'bldg_type', 'block', 'borough', 'city', 'community_board', 'dobrundate', 'expiration_date', 'filing_date', 'filing_status', 'gis_census_tract', 'gis_council_district', 'gis_latitude', 'gis_longitude', 'gis_nta_name', 'house__', 'issuance_date', 'job__', 'job_doc___', 'job_start_date', 'job_type', 'lot', 'non_profit', 'owner_s_business_name', 'owner_s_business_type', 'owner_s_zip_code', 'permit_sequence__', 'permit_si_no', 'permit_status', 'permit_type', 'permittee_s_license__', 'permittee_s_license_type', 'permittee_s_phone__', 'residential', 'self_cert', 'state', 'work_type', 'zip_code'])

In [127]:
cleaned_dict[0:5]

[{'bin__': '1077346',
  'bldg_type': '2',
  'block': '00535',
  'borough': 'MANHATTAN',
  'city': 'NEW YORK',
  'community_board': '102',
  'dobrundate': '2019-12-05T00:00:00.000',
  'expiration_date': '2020-12-03T00:00:00.000',
  'filing_date': '2019-12-04T00:00:00.000',
  'filing_status': 'INITIAL',
  'gis_census_tract': '5501',
  'gis_council_district': '1',
  'gis_latitude': '40.729267',
  'gis_longitude': '-73.996006',
  'gis_nta_name': 'West Village',
  'house__': '40',
  'issuance_date': '2019-12-04T00:00:00.000',
  'job__': '110429536',
  'job_doc___': '02',
  'job_start_date': '2019-12-04T00:00:00.000',
  'job_type': 'A2',
  'lot': '00001',
  'non_profit': 'N',
  'owner_s_business_name': 'NEW YORK UNIVERSITY',
  'owner_s_business_type': 'CORPORATION',
  'owner_s_zip_code': '10012',
  'permit_sequence__': '01',
  'permit_si_no': '3717990',
  'permit_status': 'ISSUED',
  'permit_type': 'PL',
  'permittee_s_license__': '0000907',
  'permittee_s_license_type': 'MP',
  'permittee_s

# MYSQL

In [113]:
import pandas as pd
import sqlalchemy
import config_aws

from sqlalchemy.exc import ResourceClosedError

In [114]:
permits = db.Table('permits', metadata, 
    db.Column('bin__', db.Integer()),
    db.Column('bldg_type', db.Integer()),
    db.Column('block', db.Integer()),
    db.Column('borough', db.String(16)),
    db.Column('city', db.String(20)),
    db.Column('community_board', db.Integer()),
    db.Column('dobrundate', db.DateTime()),
    db.Column('expiration_date', db.DateTime()),
    db.Column('filing_date', db.DateTime()),
    db.Column('filing_status', db.String(20)),
    db.Column('gis_census_tract', db.Integer()),
    db.Column('gis_council_district', db.Integer()),
    db.Column('gis_latitude', db.Integer()),
    db.Column('gis_longitude', db.Integer()),
    db.Column('gis_nta_name', db.String(40)),
    db.Column('house__', db.Integer()),
    db.Column('issuance_date', db.DateTime()),
    db.Column('job__', db.Integer()),
    db.Column('job_doc___', db.Integer()),
    db.Column('job_start_date', db.DateTime()),
    db.Column('job_type', db.String(15)),
    db.Column('lot', db.Integer()),
    db.Column('non_profit', db.String(20)),
    db.Column('owner_s_business_name', db.String(40)),
    db.Column('owner_s_business_type', db.String(40)),
    db.Column('owner_s_zip_code', db.String(20)),
    db.Column('permit_sequence__', db.Integer()),
    db.Column('permit_si_no', db.Integer()),
    db.Column('permit_status', db.String(20)),
    db.Column('permit_type', db.String(10)),
    db.Column('permittee_s_business_name', db.String(40)),
    db.Column('permittee_s_license__', db.Integer()),
    db.Column('permittee_s_license_type', db.String(10)),
    db.Column('residential', db.String(5)),
    db.Column('self_cert', db.String(5)), 
    db.Column('work_type', db.String(5)),
    db.Column('zip_code', db.Integer()),
    db.Column('id', db.Integer(), autoincrement=True, primary_key=True)
)

InvalidRequestError: Table 'permits' is already defined for this MetaData instance.  Specify 'extend_existing=True' to redefine options and columns on an existing Table object.

# Insert data

In [128]:
metadata.create_all(engine)

In [129]:
query = db.insert(permits)

# values = [{'bin__': 1, 'borough': 'Brooklyn'},{'bin__': 2, 'borough': 'Staten Island'}]
 
connection.execute(query, cleaned_dict)

# Keeping just in case:

In [245]:
#create database 
from mysql.connector import errorcode
db_name = 're_permits'

def create_database(cursor, database):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

try:
    cursor.execute("USE {}".format(db_name))
except mysql.connector.Error as err:
    print("Database {} does not exists.".format(db_name))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor, db_name)
        print("Database {} created successfully.".format(db_name))
        cnx.database = db_name
    else:
        print(err)
        exit(1)
        
create_database(cursor, db_name)

Failed creating database: 1007 (HY000): Can't create database 're_permits'; database exists


 Part 1 : Set up a database connection and utilities available throughout

In [246]:
db_name = 're_permits'

create_query = """
CREATE TABLE permits (
          bin__ INT,
          bldg_type INT,
          block INT,
          borough VARCHAR (20),
          city VARCHAR (20),
          community_board INT,
          dobrundate DATETIME,
          expiration_date DATETIME,
          filing_date DATETIME,
          filing_status VARCHAR (20),
          gis_census_tract INT,
          gis_council_district INT,
          gis_latitude INT,
          gis_longitude INT,
          gis_nta_name VARCHAR (40),
          house__ INT,
          issuance_date DATETIME,
          job__ INT (20),
          job_doc___ INT,
          job_start_date DATETIME,
          job_type VARCHAR (15),
          lot INT,
          non_profit VARCHAR (20),
          owner_s_business_name VARCHAR (40),
          owner_s_business_type VARCHAR (40),
          owner_s_zip_code VARCHAR (20),
          permit_sequence__ INT,
          permit_si_no INT,
          permit_status VARCHAR (20),
          permit_type VARCHAR (10),
          permittee_s_business_name VARCHAR (40),
          permittee_s_license__ INT,
          permittee_s_license_type VARCHAR (10),
          residential VARCHAR (5),
          self_cert VARCHAR (5), 
          work_type VARCHAR (5),
          zip_code INT,
          primary_key INT AUTO_INCREMENT PRIMARY KEY);"""

cursor.execute(create_query)

In [248]:
def insert_data(data):
    
    permit_tuples = [(
        df['bin__'],
        df['bldg_type'],
        df['block'],
        df['borough'],
        df['city'],
        df['community_board'],
        df['dobrundate'],
        df['expiration_date'],
        df['filing_date'],
        df['filing_status'],
        df['gis_census_tract'],
        df['gis_council_district'],
        df['gis_latitude'],
        df['gis_longitude'],
        df['gis_nta_name'],
        df['house__'],
        df['issuance_date'],
        df['job__'],
        df['job_doc___'],
        df['job_start_date'],
        df['job_type'],
        df['lot'],
        df['non_profit'],
        df['owner_s_business_name'],
        df['owner_s_business_type'],
        df['owner_s_zip_code'],
        df['permit_sequence__'],
        df['permit_si_no'],
        df['permit_status'],
        df['permit_type'],
        df['permittee_s_business_name'],
        df['permittee_s_license__'],
        df['permittee_s_license_type'],
        df['residential'],
        df['self_cert'],
        df['work_type'],
        df['zip_code']) for df in data]
        
    insert_statement = """
        INSERT INTO permits
        (bin__,
        bldg_type,
        block,
        borough,
        city,
        community_board,
        dobrundate,
        expiration_date,
        filing_date,
        filing_status,
        gis_census_tract,
        gis_council_district,
        gis_latitude,
        gis_longitude,
        gis_nta_name,
        house__,
        issuance_date,
        job__,
        job_doc___,
        job_start_date,
        job_type,
        lot,
        non_profit,
        owner_s_business_name,
        owner_s_business_type,
        owner_s_zip_code,
        permit_sequence__,
        permit_si_no,
        permit_status,
        permit_type,
        permittee_s_business_name,
        permittee_s_license__,
        permittee_s_license_type,
        residential,
        self_cert,
        work_type,
        zip_code)
        
        VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, 
        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, 
        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"""

    cursor.executemany(insert_statement,permit_tuples)
    db.commit()
    
insert_data(cleaned_dict)
                                

InterfaceError: Failed executing the operation; Not all parameters were used in the SQL statement

150